## **Import important Libraries**

In [27]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [28]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#### Device Configuration

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Hyper Parameters

In [30]:
num_epochs = 4
batch_size = 4
learning_rate = 0.001

#### Load Dataset -- cifar10

In [32]:
# dataset has PILImage images of range [0, 1]
# transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                           shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog',
           'horse', 'ship', 'truck')


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#### Convolutional Neural Network

In [33]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # No actovation function, no softmax activation function
        return x

model = ConvNet().to(device)

#### Loss and Optimizer

In [34]:
criterion = nn.CrossEntropyLoss() # have softmax
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#### Training Loop

In [36]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/4], Step [2000/12500], Loss: 2.2893
Epoch [1/4], Step [4000/12500], Loss: 2.3044
Epoch [1/4], Step [6000/12500], Loss: 2.3036
Epoch [1/4], Step [8000/12500], Loss: 2.0904
Epoch [1/4], Step [10000/12500], Loss: 2.1373
Epoch [1/4], Step [12000/12500], Loss: 2.0140
Epoch [2/4], Step [2000/12500], Loss: 1.8634
Epoch [2/4], Step [4000/12500], Loss: 2.1151
Epoch [2/4], Step [6000/12500], Loss: 2.1292
Epoch [2/4], Step [8000/12500], Loss: 1.7353
Epoch [2/4], Step [10000/12500], Loss: 1.0917
Epoch [2/4], Step [12000/12500], Loss: 2.3766
Epoch [3/4], Step [2000/12500], Loss: 1.5781
Epoch [3/4], Step [4000/12500], Loss: 1.3673
Epoch [3/4], Step [6000/12500], Loss: 1.4734
Epoch [3/4], Step [8000/12500], Loss: 1.3258
Epoch [3/4], Step [10000/12500], Loss: 1.2298
Epoch [3/4], Step [12000/12500], Loss: 1.9459
Epoch [4/4], Step [2000/12500], Loss: 1.9158
Epoch [4/4], Step [4000/12500], Loss: 1.3868
Epoch [4/4], Step [6000/12500], Loss: 2.2922
Epoch [4/4], Step [8000/12500], Loss: 1.3196
Epoc

#### Testing Loop

In [37]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        # max returns (value, index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

acc = 100.0 * n_correct / n_samples
print(f'Accuracy of the network: {acc}%')

for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc}%')

Accuracy of the network: 45.73%
Accuracy of plane: 55.7%
Accuracy of car: 61.1%
Accuracy of bird: 23.7%
Accuracy of cat: 20.9%
Accuracy of deer: 31.1%
Accuracy of dog: 44.4%
Accuracy of frog: 64.0%
Accuracy of horse: 50.7%
Accuracy of ship: 51.0%
Accuracy of truck: 54.7%
